## Plan wykładu

+ Uruchamianie programów zewnętrznych
+ Wywoływanie C
+ Zanurzanie Julii w innych programach
+ Tworzenie własnych pakietów
+ JuliaCon

## Uruchamianie programów zewnętrznych

https://docs.julialang.org/en/v0.6.0/manual/running-external-programs/

In [1]:
cmd =`python -c "print 'dwa plus dwa to {}'.format(2+2)"`

`python -c "print 'dwa plus dwa to {}'.format(2+2)"`

- Zamiast natychmiast uruchamiać polecenie, Julia tworzy obiekt Cmd do reprezentowania polecenia.
- Można użyć tego obiektu do połączenia polecenia z innymi (pipe), uruchomienia itp.
- Gdy polecenie jest uruchamiane,  wyjście domyślnie przechodzi do STDOUT
- Polecenie nie jest uruchamiane przez powłokę,  Julia analizuje składnię poleceń bezpośrednio, odpowiednio interpolując zmienne 
- Polecenie jest uruchamiane jako proces potomny, używając wywołań fork i exec.



In [2]:
typeof(cmd)

Cmd

In [6]:
# wynik działania jest wysyłany na STDOUT
ret=run(cmd)

dwa plus dwa to 4


In [7]:
# funkcja run nic nie zwraca
ret

In [8]:
# wczytanie outputu do zmiennej
a = readstring(cmd)

"dwa plus dwa to 4\n"

In [11]:
typeof(a)

String

In [5]:
# obcięcie końca linii.
chomp(a)

"dwa plus dwa to 4"

In [72]:
# open() służy to komunikacji z wejściem/wyjściem danej komendy  
open(`less`, "w", STDOUT) do io
           for i = 1:3
               println(io, i)
           end
       end

1
2
3


### Interpolacja

In [12]:
file = "/etc/passwd"

"/etc/passwd"

In [13]:
# interpolacja
cmd = `sort $file`

`sort /etc/passwd`

In [14]:
file1 = "/Volumes/External HD/data.csv"

"/Volumes/External HD/data.csv"

In [15]:
# Julia wstawia znaki '' , aby pokazać użytkownikowi, że zadbała o spację w nazwie pliku. 
`sort $file1`

`sort '/Volumes/External HD/data.csv'`

In [16]:
# powyższe działa też w przypadku iterpolacji
path = "/Volumes/External HD"
name = "data"
ext = "csv"
`sort $path/$name.$ext`

`sort '/Volumes/External HD/data.csv'`

In [17]:
#jeśli chcemy więcej plików - robimy tablicę
files = ["/etc/passwd","/Volumes/External HD/data.csv"]
`grep foo $files`

`grep foo /etc/passwd '/Volumes/External HD/data.csv'`

In [ ]:
names = ["foo","bar","baz"]

In [19]:
exts = ["aux","log"]

2-element Array{String,1}:
 "aux"
 "log"

In [37]:
# iloczyn kartezjański zbioru names i exts`ls -f $names.$exts`)
`ls -lf $names.$exts`

`ls -lf foo.aux foo.log bar.aux bar.log baz.aux baz.log`

In [21]:
# użycie pipe 
run(pipeline(`cut -d: -f3 /etc/passwd`, `sort -n`, `tail -n5`))

104
105
106
8000
65534


In [30]:
#równoległe uruchamianie komend
run(`echo hello` & `echo world`)

hello
world


In [31]:
# podłączenie outputu równolegle uruchomionych komend do inputu następnej komendy poprzez pipe
run(pipeline(`echo world` & `echo hello`, `sort`))

hello
world


## Wywoływanie funkcji  C w Julii

Julia ma filozofię "no boilerplate": 

- funkcje mogą być wywoływane bezpośrednio z Julii bez jakiegokolwiek kodu "kleju", generowania kodu lub kompilacji - nawet  w trybie interaktywnym.

- Osiąga się to poprzez odpowiednie wywołanie z użyciem składni ccall (), która wygląda jak zwykłe wywołanie funkcji.

- Kod, który ma zostać wywołany, musi być dostępny jako biblioteka współdzielona ( np. opcja -shared  -fPIC) 

- Instrukcje maszyn generowane przez kompilator JIT  są takie same, jak natywne wywołanie C (narzut jest taki sam)   

- Biblioteki i funkcje współdzielone są odwoływane przez krotkę formularza (:function, "library") lub ("function", "library"), gdzie funkcja jest nazwą wywoływaną przez C. 

- Nazwa funkcji może być użyta samodzielnie w miejsce krotki (:funkcja lub "funkcja"). W takim przypadku nazwa funkcji bedzie wzięta z  bieżącego procesu.

- można też  użyć ccall(), aby faktycznie wygenerować wywołanie funkcji biblioteki

In [67]:
#(:Funkcja, biblioteka), typ zwracany, krotka typów wejściowych , argumenty
t = ccall( (:clock, "libc"), Int32, ())

14807193

In [68]:
typeof(t)

Int32

In [69]:
# musi być  (Cstring,), a nie  (Cstring) żeby uzyskać krotkę
shell = ccall((:getenv, "libc"), Cstring, (Cstring,), "SHELL")

Cstring(0x00007ffc75e9bc82)

In [70]:
typeof(shell)

Cstring

In [71]:
# konwersja na Juliowy String. unsafe t.j. zakończy się błędem, jeśli wskaźnik nie wskazuje na co powinien . 
shell_in_julia=unsafe_string(shell)

"/bin/bash"

In [72]:
typeof(shell_in_julia)

String

In [73]:
# zwykle opakowuje się ccall tak, aby obsłużyc błedy.
# Błedy w różnych bibliotekach C są zwracane w różny sposób -może być NULL, może byc -1 itd.
function getenv(var::AbstractString)
  val = ccall((:getenv, "libc"),
              Cstring, (Cstring,), var)
  if val == C_NULL
    error("getenv: undefined variable: ", var)
  end
  unsafe_string(val)
end


getenv (generic function with 1 method)

In [74]:
getenv("PATH")

"/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"

In [75]:
getenv("FOOBAR")

LoadError: [91mgetenv: undefined variable: FOOBAR[39m

## Zanurzanie w C - przykład

https://docs.julialang.org/en/v0.6.2/manual/embedding/

```c
#include <julia.h>

int main(int argc, char *argv[])
{
    /* required: setup the Julia context */
    jl_init(NULL);

    /* run Julia commands */
    jl_eval_string("print(sqrt(2.0))");

    /* strongly recommended: notify Julia that the
         program is about to terminate. this allows
         Julia time to cleanup pending write requests
         and run all finalizers
    */
    jl_atexit_hook(0);
    return 0;
}
}
```
---

```
gcc -o test -fPIC -I$JULIA_DIR/include/julia -L$JULIA_DIR/lib test.c -ljulia $JULIA_DIR/lib/julia/libstdc++.so.6

```
---



## Wywoływanie kodu z Pythona

Pakiet PyCall  https://github.com/JuliaPy/PyCall.jl

```julia
pkg.add("PyCall")
```

In [76]:
using PyCall

INFO: Precompiling module Conda.
INFO: Precompiling module PyCall.


In [77]:
# importujemy moduł Pythonowy
@pyimport numpy as np

In [78]:
a = np.array([1,2,3,4])

4-element Array{Int64,1}:
 1
 2
 3
 4

In [79]:
typeof(a)

Array{Int64,1}

In [80]:
np.sin(a)

4-element Array{Float64,1}:
  0.841471
  0.909297
  0.14112 
 -0.756802

In [81]:
pyeval("[i**2 for i in range(10)]")

10-element Array{Int64,1}:
  0
  1
  4
  9
 16
 25
 36
 49
 64
 81

In [82]:
@pyimport os

In [83]:
# różnica  wołamy foo[:bar] oraz foo[:bar](...) a nie foo.bar and foo.bar(...)
os.sys.path

LoadError: [91mtype PyObject has no field path[39m

In [84]:
os.sys[:path]

7-element Array{String,1}:
 "/usr/lib/python2.7"                      
 "/usr/lib/python2.7/plat-x86_64-linux-gnu"
 "/usr/lib/python2.7/lib-tk"               
 "/usr/lib/python2.7/lib-old"              
 "/usr/lib/python2.7/lib-dynload"          
 "/usr/local/lib/python2.7/dist-packages"  
 "/usr/lib/python2.7/dist-packages"        

## Pakiety

https://docs.julialang.org/en/v0.6.2/manual/packages/

- Pakiety Julia to po prostu repozytoria git

- Oficjalne pakiety Julia są zarejestrowane w repozytorium METADATA.jl, dostępne w dobrze znanej lokalizacji
https://github.com/JuliaLang/METADATA.jl Zawiera ono informacje o linku do każdego pakietu oraz jego wersji.

- użytkownicy mogą tworzyć swoje własne repozytoria METADATA. Pozwala to na wybieranie, które pakiety są dostępne do automatycznej instalacji. Można dopuścić tylko sprawdzone i zatwierdzone wersje pakietów albo udostępniać prywatne pakiety

- Polecenia Pkg.add() i Pkg.rm() współdziałają z zarejestrowanymi pakietami

- menedżer pakietów może również instalować i pracować z niezarejestrowanymi pakietami. Aby zainstalować niezarejestrowany pakiet, używa się Pkg.clone(URL), gdzie URL to git URL, z którego pakiet można sklonować.

```julia
Pkg.clone("git://example.com/path/to/Package.jl.git")
```

- każdy użytkownik ma lokaną kopię METADATA w katalogu .julia/VERSION/METADATA np. /home/user/.julia/v0.6/METADATA

- Pakiet do tworzenia i rejestrowanie pakietów

```julia
Pkg.add("PkgDev")
```

In [3]:
using PkgDev

In [1]:
# tworzymy pakiet lokalnie 
# jeśli mamy skonfigurowany dostęp do githuba poprzez gita 
# (https://help.github.com/articles/connecting-to-github-with-ssh/) PkgDev wygeneruje dla nas url
# nie wygeneruje zdalnego repo  na Githubie (trzeba to zrobić ręcznie)

PkgDev.generate("AGH_Julia_Lab2","MIT", force=true)

INFO: Initializing AGH_Julia_Lab2 repo: C:\Users\kzajac\AppData\Local\JuliaPro-0.6.2.2w\pkgs-0.6.2.2\v0.6\AGH_Julia_Lab2
INFO: Generating LICENSE.md
INFO: Generating README.md
INFO: Generating src/AGH_Julia_Lab2.jl
INFO: Generating test/runtests.jl
INFO: Generating REQUIRE
INFO: Generating .gitignore
INFO: Generating .travis.yml
INFO: Generating appveyor.yml
INFO: Generating .codecov.yml
INFO: Committing AGH_Julia_Lab2 generated files


Zawartość wygenerowanego pakietu:

```console
kzajac@tabaluga MINGW64 ~/AppData/Local/JuliaPro-0.6.2.2w/pkgs-0.6.2.2/v0.6/AGH_Julia_Lab2 (master)
$ ls -la
total 88
drwxr-xr-x 1 kzajac 197121    0 cze 13 14:29 ./
drwxr-xr-x 1 kzajac 197121    0 cze 13 14:29 ../
-rw-r--r-- 1 kzajac 197121   15 cze 13 14:29 .codecov.yml
drwxr-xr-x 1 kzajac 197121    0 cze 14 10:09 .git/
-rw-r--r-- 1 kzajac 197121   29 cze 13 14:29 .gitignore
-rw-r--r-- 1 kzajac 197121 1130 cze 13 14:29 .travis.yml
-rw-r--r-- 1 kzajac 197121 2148 cze 13 14:29 appveyor.yml
-rw-r--r-- 1 kzajac 197121 1209 cze 13 14:29 LICENSE.md
-rw-r--r-- 1 kzajac 197121   17 cze 13 14:29 README.md
-rw-r--r-- 1 kzajac 197121   10 cze 13 14:29 REQUIRE
drwxr-xr-x 1 kzajac 197121    0 cze 13 14:29 src/
drwxr-xr-x 1 kzajac 197121    0 cze 13 14:29 test/
```


Zakladamy ręcznie repozytorium  na githubie i  przesylamy do niego zawartość wygenerowanego pakietu :
```console
kzajac@tabaluga MINGW64 ~/AppData/Local/JuliaPro-0.6.2.2w/pkgs-0.6.2.2/v0.6/AGH_Julia_Lab2 (master)
$ git remote add origin git@github.com:kzajac/AGH_Julia_Lab2.jl.git

kzajac@tabaluga MINGW64 ~/AppData/Local/JuliaPro-0.6.2.2w/pkgs-0.6.2.2/v0.6/AGH_Julia_Lab2 (master)
$ git push -u origin master
Warning: Permanently added the RSA host key for IP address '192.30.253.113' to the list of known hosts.
Counting objects: 15, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (8/8), done.
Writing objects: 100% (15/15), 3.12 KiB | 533.00 KiB/s, done.
Total 15 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), done.
To github.com:kzajac/AGH_Julia_Lab2.jl.git
 * [new branch]      master -> master
Branch master set up to track remote branch master from origin.
```

Mozna ogladac na :
https://github.com/kzajac/AGH_Julia_Lab2.jl

In [4]:
# po zakomitowaniu zmian możemy zarejestrować nasz pakiet w lokalnym METADATA
PkgDev.register("AGH_Julia_Lab2")

INFO: Registering AGH_Julia_Lab2 at https://github.com/kzajac/AGH_Julia_Lab2.jl.git
INFO: Committing METADATA for AGH_Julia_Lab2


Zawartość podkatalogu METADATA dotycząca naszego pakietu:

```console
kzajac@tabaluga MINGW64 ~/AppData/Local/JuliaPro-0.6.2.2w/pkgs-0.6.2.2/v0.6/METADATA/AGH_Julia_Lab2 (metadata-v2)
$ ls
url

kzajac@tabaluga MINGW64 ~/AppData/Local/JuliaPro-0.6.2.2w/pkgs-0.6.2.2/v0.6/METADATA/AGH_Julia_Lab2 (metadata-v2)
$ cat url
https://github.com/kzajac/AGH_Julia_Lab2.jl.git
```


Można otagować numerem wersji
```julia
PkgDev.tag("AGH_Julia_Lab2")
```

```console
kzajac@tabaluga MINGW64 ~/AppData/Local/JuliaPro-0.6.2.2w/pkgs-0.6.2.2/v0.6/METADATA/AGH_Julia_Lab2 (metadata-v2)
$ ls
url  versions/

kzajac@tabaluga MINGW64 ~/AppData/Local/JuliaPro-0.6.2.2w/pkgs-0.6.2.2/v0.6/METADATA/AGH_Julia_Lab2 (metadata-v2)
$ ls versions/
0.0.1/
```


Jeśli chcemy, aby nasz pakiet stał się oficjalnym pakietem należy zarejestrować go w oficjalnym repo METADATA poprzez:
Można to zrobić ręcznie albo  komendą  
```julia
PkgDev.publish()
``` 

Można też to zrobić ręcznie  poprzez wykonanie kroków:

- zrobic fork repozytorium METADATA  na własne konto na Githubie 
- skonfigurować je jako repozytorium zdalne dla lokalnego repozytorium METADATA
- zrobić operacje push do tego zdalnego repozytorium swoich zmian w lokalnym METADATA  
- otworzyć  pull request do centralnego METADATA




## JuliaCon
http://juliacon.org/2018/


Ciekawe Pakiety :

- Flux machine learning with Julia http://fluxml.ai/Flux.jl/stable/
- Analiza języków naturalnych https://github.com/JuliaText/TextAnalysis.jl
- Query http://www.david-anthoff.com/Query.jl/stable/querycommands.html
- Problemy optymalizacyjne (programowanie z ograniczeniami) 
    * https://github.com/JuliaOpt/juliaopt-notebooks/blob/master/notebooks/Shuvomoy%20-%20Getting%20started%20with%20JuMP.ipynb
    * https://github.com/JuliaOpt/juliaopt-notebooks/blob/master/notebooks/JuMP-Sudoku.ipynb
- Celeste - analiza obrazów z ciałami niebieskimi

    - ciała niebieskie dostarczają fotonów, które składają się na wyprodukowanie pixela w obrazku

    - duże dane (teleskopy produkują setki GB  na dobę, będą produkować TB )

    - https://github.com/jeff-regier/Celeste.jl